<a href="https://colab.research.google.com/github/carinunez/ControlNet/blob/main/ControlNet_try2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ControlNet

Integrantes:
- xx
- xx
- xx
-xx

Arquitectura neuronal utilizada para agregar controles/restricciones especiales a los modelos de difusión de texto a imágenes preentrenados. En este caso, la arquitectura se aplico a Stable Diffusion, creando una copia entrenable de la UNet en el encoder que se une al modelo original utilizando zero-convolutions.


Escoger qué condiciones se probarán-> pose, profundidad,


In [ ]:
import torch
import torch.nn as nn
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import StableDiffusionPipeline, DDPMScheduler, UNet2DConditionModel
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, Dataset
from PIL import Image
import torchvision.transforms as T


In [ ]:
device = "cuda" if torch.cuda.is_available() else None
device

In [ ]:
model_id= "runwayml/stable-diffusion-v1-5"

# 1. Carga de los datos


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2
import torchvision.transforms as T


# 1.2 Generación de Condiciones con Canny

In [ ]:
image_dir = "/content/drive/MyDrive/img_pkmn"
canny_dir  ="/content/drive/MyDrive/canny"

def apply_canny_path(image: Image.Image, im_path, path=False) -> Image.Image:
    if path:
      img = np.array(image.convert("RGB"))
      edges = cv2.Canny(img, 100, 200)
      edges = np.stack([edges] * 3, axis=-1)  # convertir a 3 canales
      edges = Image.fromarray(edges)
      edges.save(os.path.join(canny_dir, f'canny_{im_path}'))
    else:
      edges = Image.open(os.path.join(canny_dir, f'canny_{im_path}'))
    return edges

if not os.path.isdir(canny_dir):
    os.makedirs(canny_dir)

data = []
for im_path in os.listdir(image_dir):
  img = Image.open(os.path.join(image_dir, im_path)).convert("RGB")
  # si path es True, se generan las imag, sino se abren y cargan
  canny = apply_canny_path(img, im_path, path=False)
  prompt = "a colorful pokemon"
  data.append({"image": img, 'canny':canny, "text": f"a colorful pokemon_{im_path}"})

dataset = Dataset.from_list(examples)

In [ ]:
data

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 12))
ax1.imshow(data[0]['image']);
ax1.set_title('Original');
ax2.imshow(data[0]['canny']);
ax2.set_title('Canny');


In [ ]:
class ControlDataset(Dataset):
    def __init__(self, dataset, size=512):
        self.dataset = dataset
        self.image_trans = T.Compose([
            T.Resize((size, size)),
            T.ToTensor(),
            T.Normalize([0.5]*3, [0.5]*3)  # RGB
        ])

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
      sample = self.dataset[idx]

      img = self.image_trans(sample['image'])
      ctrl = self.image_trans(sample['canny'])
      prompt = sample[idx]
      return img, ctrl, prompt


In [ ]:
# Dataloader
dataset = Dataset.from_list(data)
train_dataloader = DataLoader(ControlDataset(dataset), batch_size=1, shuffle=True)

Se agrega un acelerador para hacer una prueba

In [ ]:
from diffusers import UNet2DConditionModel, AutoencoderKL, DDIMScheduler
from transformers import CLIPTextModel
import torch

# Carga el UNet preentrenado
unet = UNet2DConditionModel.from_pretrained(
    model_id, subfolder="unet"
).to(device)

# Carga el VAE preentrenado
vae = AutoencoderKL.from_pretrained(
    model_id, subfolder="vae"
).to(device)

# Carga el text encoder CLIP
text_encoder = CLIPTextModel.from_pretrained(
    "openai/clip-vit-large-patch14"
).to(device)

# Scheduler
scheduler = DDIMScheduler.from_pretrained(
    model_id, subfolder="scheduler"
)

# 2. Tokenizador

Importamos el tokenizador de Stable Diffusion para evitar problemas/inconsistencias con el tamaño de los embeddings del texto c/r a la unet

In [ ]:
tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder='tokenizer')
text_encoder = CLIPTextModel.from_pretrained(model_id, subfolder='text_encoder')

def get_text_embedding(prompt):
  inputs = tokenizer(prompt, padding='max_length', trunctation=True,
                     max_length=77, return_tensors='pt')

  with torch.no_grad():
    return text_encoder(**inputs.to('cuda')).last_hidden_state

# 2. Bloques de ControlNet

El bloque de ControlNet toma:
- x: imagen ruidosa
- t: paso de difusión
- encoder_hidden_states: embeddings del texto
- control_im: imagen utilizada para condicionar

Se generan activaciones residuales de control_im con las zeroConv2d, luego inyecto estas activaciones a los bloques originales de Stable Diffusion
Finalmente, se entrena manteniendo congelada la capa original de UNet

In [ ]:
# Forma de inyectar la condicion a StableDifussion
class zeroConv2d(nn.Conv2d):
    def __init__(self, in_c, out_c, kernel_size=1, stride=1, padding=0):
        super().__init__(in_c, out_c, kernel_size, stride, padding)
        nn.init.zeros_(self.weight)
        if self.bias is not None:
            nn.init.zeros_(self.bias)

In [ ]:
class MYControlNet(nn.Module):
  def __init__(self, unet:UNet2DConditionModel):
    super().__init__()
    # .eval para congelar los pesos i.e no se entrenan
    self.unet = unet.eval()

    # copia de Stable Diffusion para entrenar
    copy = unet.config
    self.cond_proj = nn.ModuleList(
        zeroConv2d(copy.in_channels, out_c, kernel_size=3, padding=1)
        # aseguro que la copia tenga la misma cantidad de canales que cada
        # cada bloque de DOWNSAMPLING
        # Con esto puedo sumar directamente las activaciones a la salida en c/bloque
        for out_c in copy.block_out_channels
    )

    # proyeccion bloque central/intermedio de la UNet -> BottleNeck
    self.mid = zeroConv2d(copy.in_channels, copy.block_out_channels[-1], kernel_size=3, padding=1)

  def forward(self, x, timesteps, encoder_hidden_states, control_im):
    # Activaciones de control
    cond_activ = [] # se guardan para usarlos como residual en los down_blocks

    for proj in self.cond_proj:
      cond = proj(control_im)
      cond_activ.append(cond)

    mid_activ = self.mid(control_im)

    # se guardan los residuos para entregarselos al decoder mediante skip-connections
    down_activ = []
    h=x
    for block, cond_res in zip(self.unet.down_blocks, cond_activ):
      h = block(h, timesteps, encoder_hidden_states)
      h = h + cond_res

    h = self.unet.mid_block(h, timesteps, encoder_hidden_states)
    h = h + mid_activ

    for block, skip in zip(self.unet.up_blocks, reversed(down_activ)):
      h = block(h, timesteps, encoder_hidden_states=skip)

    return self.unet.out(h)



# 4.1 Entrenamiento pipeline

In [ ]:
from diffusers import StableDiffusionPipeline

class MyPipeline(StableDiffusionPipeline):
    def __init__(self, vae, text_encoder, tokenizer, my_controlnet):
        super().__init__(vae=vae, text_encoder=text_encoder, tokenizer=tokenizer, unet=None)
        self.my_controlnet = my_controlnet

    def forward(self, *args, **kwargs):
        # Aquí deberías adaptar el forward para llamar a tu my_controlnet.forward
        pass

    def __call__(self, prompt, control_image, **kwargs):
        # Tokenizar texto
        inputs = self.tokenizer(prompt, return_tensors="pt", padding="max_length", truncation=True, max_length=77)
        encoder_hidden_states = self.text_encoder(**inputs).last_hidden_state

        # Preparar latentes, timesteps, etc. (similar a pipeline normal)
        latents = vae.encode(imgs).latent_dist.sample() * 0.18215
        noise = torch.rand_like(img)
        steps = torch.randoint(0, 1000, (img.shape[0],), device).long()
        img_w_noise = noise_sch.add_noise(img, noise, steps)

        # Usar MYControlNet para hacer la inferencia
        output = self.my_controlnet(x=latents, timesteps=timesteps,
                                    encoder_hidden_states=encoder_hidden_states,
                                    control_im=control_image)

        # Decodificar latentes con VAE, etc.
        # ...

        return output


# 4. Entrenamiento

In [ ]:
# scheduler
noise_sch = DDPMScheduler(num_train_timesteps=1000)

In [ ]:
def train_CN(model, dataloader, optimizer, lr_scheduler,
             epochs=10, lr=1e-4, save_each=3,
             device='cuda'):
  model.to('cuda')

  for epoch in range(epochs):
    # model.train()
    for i, (img, control_im, prompt) in enumerate(dataloader):
      img = img.to(device)
      ctrl_img = ctrl_img.to(device)
      text_emb = get_text_embedding(prompt)

      # inicializo el ruido
      noise = torch.rand_like(img)
      steps = torch.randoint(0, 1000, (img.shape[0],), device='cuda').long()
      img_w_noise = noise_sch.add_noise(img, noise, steps)

      # foward
      pred = model(img_w_noise, steps, text_emb, control_im)

      # loss
      loss = F.mse_loss(pred, noise)

      accelerator.backward(loss)
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()

      if (i+1) %10 == 0:
        print(f'Epoca: {epoch}/epochs Step {steps} Loss: {loss.item():.4f}')

      # Checkpoints
      if epoch % save_each == 0:
        accelerator.save_model(model, f"./checkpoints/epoch_{epoch}")


In [ ]:
# Congelo la UNet
model = MYControlNet(unet).to(device)
for param in model.unet.parameters():
  param.requires_grad = False

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator(
    mixed_precision="fp16",  # o bf16
    log_with="tensorboard",  # o wandb
    project_dir="./logs")

optimizer = torch.optim.AdamW(
                  filter(lambda p: p.requires_grad, model.parameters()),
                  lr=1e-4)

# usado para un aprendizaje más suave
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=1000)

model, optimizer, dataloader, lr_scheduler = accelerator.prepare(model, optimizer,
                                                                 train_dataloader, lr_scheduler)


In [ ]:
from diffusers import StableDiffusionControlNetPipeline
from transformers import CLIPFeatureExtractor

feature_extractor = CLIPFeatureExtractor.from_pretrained(model_id, subfolder="feature_extractor")


In [ ]:
pipe = StableDiffusionControlNetPipeline(
    vae=vae,
    text_encoder=text_encoder,
    tokenizer=tokenizer,
    unet=unet,
    controlnet=ControlNet2,
    safety_checker=None,
    scheduler=scheduler,
    feature_extractor=feature_extractor)

# 2ND try

In [ ]:
# Asumiendo que tienes tu dataset con (input_latents, timesteps, encoder_hidden_states, control_images, target_latents)

my_controlnet = MYControlNet(unet)

optimizer = torch.optim.Adam(my_controlnet.parameters(), lr=1e-4)

for epoch in range(10):
    for batch in dataloader:
        input_latents = batch["latents"].to(device)
        timesteps = batch["timesteps"].to(device)
        encoder_hidden_states = batch["encoder_hidden_states"].to(device)
        control_images = batch["control_images"].to(device)
        target = batch["target"].to(device)

        optimizer.zero_grad()

        output = my_controlnet(input_latents, timesteps, encoder_hidden_states, control_images)

        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
